In [21]:
# 1. Import packages

import os, mlflow
import pandas as pd
import numpy as np

from sklearn import model_selection, preprocessing, metrics
from sklearn.ensemble import RandomForestClassifier

In [22]:
# Data loading
os.chdir(r"C:\Users\MSFRotPC04\Desktop\CIAST_ML_Siri 2\ml-project-1")
print(os.getcwd())

C:\Users\MSFRotPC04\Desktop\CIAST_ML_Siri 2\ml-project-1


In [23]:
# Load the data

df = pd.read_csv(r"datasets\titanic\train.csv")

print(df.info())
print("_"*100)
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
____________________________________________________________________________________________________
       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000  

In [24]:
#print(df.columns)

In [25]:
# EDA
# Find the values count Name
print(df['Name'].value_counts())

Name
Braund, Mr. Owen Harris                     1
Boulos, Mr. Hanna                           1
Frolicher-Stehli, Mr. Maxmillian            1
Gilinski, Mr. Eliezer                       1
Murdlin, Mr. Joseph                         1
                                           ..
Kelly, Miss. Anna Katherine "Annie Kate"    1
McCoy, Mr. Bernard                          1
Johnson, Mr. William Cahoone Jr             1
Keane, Miss. Nora A                         1
Dooley, Mr. Patrick                         1
Name: count, Length: 891, dtype: int64


In [26]:
# Name, PassangerId, Ticket are not useful, so we will drop them

df =df.drop(["Name", "PassengerId", "Ticket", "Cabin"], axis=1)

In [27]:
# Checking missing value
print(df.isna().sum())

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64


In [28]:
# Fill the massing values in Embarked with mode value
df = df.fillna({'Embarked': df['Embarked'].mode()[0]})
print(df.isna().sum())

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      0
dtype: int64


In [29]:
#Fill missing value with -0.5 for unknown ages
df["Age"] = df["Age"].fillna(-0.5)
print(df.isna().sum())

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64


In [30]:
age_range = [-1, 0, 5, 12, 18, 24, 35, 60, np.inf] #9
labels_group =["Unknown","Baby","Child","Teenager","Student","Young Adult","Adult","Senior"]#bins-1
df["AgeGroup"] = pd.cut(df["Age"], bins=age_range, labels=labels_group)
print(df['AgeGroup'].head)

<bound method NDFrame.head of 0          Student
1            Adult
2      Young Adult
3      Young Adult
4      Young Adult
          ...     
886    Young Adult
887        Student
888        Unknown
889    Young Adult
890    Young Adult
Name: AgeGroup, Length: 891, dtype: category
Categories (8, object): ['Unknown' < 'Baby' < 'Child' < 'Teenager' < 'Student' < 'Young Adult' < 'Adult' < 'Senior']>


In [31]:
# Drop the original age column

df =df.drop(["Age"], axis=1)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   Survived  891 non-null    int64   
 1   Pclass    891 non-null    int64   
 2   Sex       891 non-null    object  
 3   SibSp     891 non-null    int64   
 4   Parch     891 non-null    int64   
 5   Fare      891 non-null    float64 
 6   Embarked  891 non-null    object  
 7   AgeGroup  891 non-null    category
dtypes: category(1), float64(1), int64(4), object(2)
memory usage: 50.1+ KB
None


In [32]:
# Check for value counts for two categorical columns
print(np.unique(df[['Sex']].values))
print(np.unique(df[['Embarked']].values))
print(np.unique(df[['AgeGroup']].values))

['female' 'male']
['C' 'Q' 'S']
['Adult' 'Baby' 'Child' 'Senior' 'Student' 'Teenager' 'Unknown'
 'Young Adult']


In [33]:
# Process the AgeGroup column with ordinal Encoder
ordinal_encoder =preprocessing.OrdinalEncoder(categories=[labels_group])
df['AgeGroup'] = ordinal_encoder.fit_transform(df['AgeGroup'].values.reshape(-1,1))
print(df['AgeGroup'].head())

0    4.0
1    6.0
2    5.0
3    5.0
4    5.0
Name: AgeGroup, dtype: float64


In [34]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  891 non-null    object 
 7   AgeGroup  891 non-null    float64
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB
None


In [35]:
# Create a 2nd Ordinal 
ordinal_encoder_2 =preprocessing.OrdinalEncoder()
df[['Sex','Embarked']] = ordinal_encoder_2.fit_transform(df[['Sex','Embarked']].values)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  891 non-null    float64
 7   AgeGroup  891 non-null    float64
dtypes: float64(4), int64(4)
memory usage: 55.8 KB
None


In [36]:
# Train test split

features = df.copy()
label = features.pop('Survived')

x_train, x_test, y_train, y_test = model_selection.train_test_split(features, label, shuffle=True, train_size=0.8,random_state=64)


In [37]:
# Feature scaling
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('scaler', preprocessing.StandardScaler()),
    ('model', RandomForestClassifier())
])

In [38]:
# Conduct the training in mlflow experiment

# mlflow.create_experiment('titanic_experiment')

mlflow.set_experiment('titanic_experiment')

mlflow.set_experiment_tag('mlflow_version', '2.15.1')


In [39]:
# Start MLflow

with mlflow.start_run():
    mlflow.sklearn.autolog()
    pipeline.fit(x_train, y_train.values)
    predictions = pipeline.predict(x_test)
    accuracy = metrics.accuracy_score(y_test.values, predictions)
    precision = metrics.precision_score(y_test.values, predictions)
    recall = metrics.recall_score(y_test.values, predictions)
    f1 = metrics.f1_score(y_test.values, predictions)

2024/08/08 16:39:13 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\MSFRotPC04\anaconda3\envs\ciast_ai01\lib\site-packages\mlflow\types\utils.py:406: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2024/08/08 16:39:15 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\MSFRotPC04\anaconda3\envs

In [40]:
import pickle as pkl

with open('src/ordinal_encoder_1.pkl', 'wb') as f:
    pkl.dump(ordinal_encoder,f)

with open('src/ordinal_encoder_2.pkl', 'wb') as f:
    pkl.dump(ordinal_encoder_2,f)